In [12]:
import json
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [13]:
def var(diagcompil,offdiagcompil,spin,region,value,error):
    diagdict = {'spin':spin, 'region':region, 'avg':0,'error':0}

    meanx = []
    errx = [] 
    totxerr = []
    totmeanx = []
    meany = []
    erry = []
    totmeany = []
    totyerr = []
    covstore = []
    coverrstore = []

    #i is row; j is column
    for (i,row) in enumerate(value):
        for (j,val) in enumerate(row):
            meanx.append(i*value[i][j])
            errx.append((i*error[i][j])**2)
        totmeanx.append(sum(meanx))
        totxerr.append(sum(errx))
        meanx=[]
        errx=[]
    for (i,row) in enumerate(value):
        for (j,val) in enumerate(row):
            meany.append(j*value[i][j])   
            erry.append((j*error[i][j])**2)
        totmeany.append(sum(meany))
        totyerr.append(sum(erry))
        meany=[]
        erry=[]

    totmeanx =sum(totmeanx)
    totxerr = math.sqrt(sum(totxerr))
    totmeany = sum(totmeany)
    totyerr = math.sqrt(sum(totyerr))
        
    offdiagdict = {'spin':spin, 'region':region, 'covar':0,'error':0}
    
    for (i,row) in enumerate(value):
        for (j,val) in enumerate(row):
            covstore.append((i- totmeanx)*(j- totmeany)*(value[i][j]))
#            covstore.append((i*j)*value[i][j]-totmeanx*totmeany*value[i][j])
            if value[i][j] > 0.001:
                coverrstore.append(i*j*error[i][j]+math.sqrt((totxerr/totmeanx)**2+(totyerr/totmeany)**2)+(error[i][j]/value[i][j])**2)
    coverrstore = [i**2 for i in coverrstore]
    totcoverr =math.sqrt(sum(coverrstore)) 
    offdiagdict['covar'] = sum(covstore)
    offdiagdict['error'] = totcoverr
#    if ((spin[0] != spin[1]) or (region[0] != region[1])):
    return offdiagcompil.append(offdiagdict)

In [47]:
data = pd.read_json("region.json")
spin = data['total_spin'][2]
hybrid = data['dft'][2]['functional']['hybrid']
results = data['qmc'][2]['postprocess']['results']

In [59]:
fmresult = data['qmc'][3]['dmc']
afmresult =data['qmc'][2]['dmc']

In [38]:
rfdata = data['qmc'][2]['postprocess']['results'][0]['results']['properties']['region_fluctuation']['fluctuation data']
rfdata = pd.DataFrame.from_dict(rfdata)

In [39]:
#Obtain full data frame -- Mn,O sites and all spins
diagcompil = []
offdiagcompil =[]
for i in range(len(rfdata)):
        var(diagcompil,offdiagcompil,rfdata['spin'][i],rfdata['region'][i],rfdata['value'][i],rfdata['error'][i])
offdiag = pd.DataFrame(offdiagcompil)
diag = pd.DataFrame(diagcompil)

#for i in range(len(offdiag)):
#    if offdiag['covar'][i]> offdiag['error'][i]:
#        print(offdiag['covar'][i],offdiag['error'][i],offdiag['region'][i],offdiag['spin'][i])

In [40]:
#Site-averaging
#0-7 correspond to Mn; 8-15 correspond to Oxygen
#Compile list for all equivalent exchanges and then average
data = pd.read_json("distances.json")
data = pd.DataFrame.from_dict(data)
data = pd.concat([data]*4, ignore_index=True)
df = pd.concat([offdiag, data], axis=1)

In [74]:
dfgroup=df.groupby(['distance'])
nearrf = dfgroup.get_group(2.22)
for i, row in nearrf.iterrows():
    newspin = int(str(row['spin'][0])+str(row['spin'][1]))
    if newspin == 10:
        newspin = 1
    nearrf.set_value(i,'spin',newspin)
neargroup = nearrf.groupby(['spin'])
nearrf = neargroup.get_group(00)
nearrf= nearrf.reset_index()
len(nearrf)

44

In [76]:
covar = []
err =[]
for i in range(len(nearrf)): 
    covar.append(nearrf['covar'][i])
    err.append(nearrf['error'][i])
covaravg = sum(covar)#/44??
err = [i**2 for i in err]
totcoverr =math.sqrt(sum(err))#/44??
print(covaravg, totcoverr)

-0.0727777737877 0.01632228347016401
